<a href="https://colab.research.google.com/github/vasan12sp/DocSummarizer/blob/master/Summ_xlnet_bart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets rouge-score torch

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.4 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=75011bd1432c3dc50336cbe56952dedcf3c9355d6c641abfc55d820f44bf949f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from datasets import load_dataset
import random

# Load CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Randomly sample 10,000 articles from the train set and 500 from the validation set
train_sample = random.sample(range(len(dataset['train'])), 10000)
val_sample = random.sample(range(len(dataset['validation'])), 500)

train_dataset = dataset['train'].select(train_sample)
val_dataset = dataset['validation'].select(val_sample)

print(f"Training on {len(train_dataset)} articles")
print(f"Evaluating on {len(val_dataset)} articles")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Training on 10000 articles
Evaluating on 500 articles


In [ ]:
from torch.utils.data import DataLoader

def preprocess_function(examples):
    inputs = examples['article']
    labels = examples['highlights']
    return {"input_text": inputs, "target_text": labels}

# Apply preprocessing to the dataset
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

# Create dataloaders
train_dataloader = DataLoader(tokenized_train, batch_size=4, shuffle=True)
val_dataloader = DataLoader(tokenized_val, batch_size=4, shuffle=False)


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AdamW
from torch.cuda.amp import GradScaler, autocast
from evaluate import load  # For evaluation metrics
from tqdm import tqdm  # For progress bar

class Summ_xlnet_bart:
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {self.device}")

        # Load XLNet
        self.xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
        self.xlnet_model = XLNetForSequenceClassification.from_pretrained(
            'xlnet-base-cased',
            num_labels=1  # Binary classification for importance
        ).to(self.device)

        # Load BART
        self.bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
        self.bart_model = BartForConditionalGeneration.from_pretrained(
            'facebook/bart-large-cnn'
        ).to(self.device)

        # Set models to training mode
        self.xlnet_model.train()
        self.bart_model.train()

        # Mixed precision training
        self.scaler = GradScaler()

    def train(self, train_dataloader, val_dataloader, epochs=3, learning_rate=5e-5):
        """
        Train both XLNet and BART models with mixed precision.

        Args:
            train_dataloader: DataLoader for the training data
            val_dataloader: DataLoader for the validation data
            epochs (int): Number of training epochs
            learning_rate (float): Learning rate for optimization
        """
        optimizer = AdamW(
            list(self.xlnet_model.parameters()) + list(self.bart_model.parameters()),
            lr=learning_rate
        )

        loss_fn = torch.nn.BCEWithLogitsLoss()  # Loss for sentence classification (XLNet)

        for epoch in range(epochs):
            print(f"Epoch {epoch+1}/{epochs}")

            total_loss = 0
            self.xlnet_model.train()
            self.bart_model.train()

            # Use tqdm for progress bar
            progress_bar = tqdm(train_dataloader, desc="Training", leave=False)

            for batch_idx, batch in enumerate(progress_bar):
                inputs = batch['input_text']
                labels = batch['target_text']

                # Tokenize inputs for XLNet
                xlnet_inputs = self.xlnet_tokenizer(
                    inputs,
                    padding=True,
                    truncation=True,
                    return_tensors='pt',
                    max_length=512
                ).to(self.device)

                # Mixed precision training with autocast
                with autocast():
                    outputs = self.xlnet_model(**xlnet_inputs)
                    importance_scores = outputs.logits.squeeze()

                    # Compute loss
                    loss = loss_fn(importance_scores, torch.ones_like(importance_scores))

                # Backpropagation with mixed precision
                optimizer.zero_grad()
                self.scaler.scale(loss).backward()
                self.scaler.step(optimizer)
                self.scaler.update()

                total_loss += loss.item()

                # Update progress bar description with the current loss and training percentage
                progress = f"Loss: {loss.item():.4f}, Progress: {100 * (batch_idx + 1) / len(train_dataloader):.2f}%"
                progress_bar.set_description(progress)

            avg_train_loss = total_loss / len(train_dataloader)
            print(f"Avg Training Loss: {avg_train_loss:.4f}")

            # Validation after every epoch
            self.evaluate(val_dataloader)

    def evaluate(self, val_dataloader):
        """
        Evaluate the model with ROUGE scores using 500 articles.

        Args:
            val_dataloader: DataLoader for the validation data
        """
        rouge = load("rouge")  # Use evaluate library to load the rouge metric
        predictions = []
        references = []

        with torch.no_grad():
            self.xlnet_model.eval()
            self.bart_model.eval()

            for batch in val_dataloader:
                inputs = batch['input_text']
                labels = batch['target_text']

                # Tokenize inputs for XLNet
                xlnet_inputs = self.xlnet_tokenizer(
                    inputs,
                    padding=True,
                    truncation=True,
                    return_tensors='pt',
                    max_length=512
                ).to(self.device)

                # Extract important sentences (Extractive step)
                outputs = self.xlnet_model(**xlnet_inputs)
                importance_scores = outputs.logits.squeeze().cpu().numpy()
                top_sentences = np.argsort(importance_scores)[-3:]

                extractive_summary = ". ".join([inputs[0].split('.')[i] for i in top_sentences]) + '.'

                # Tokenize extractive summary for BART
                bart_input = self.bart_tokenizer(
                    extractive_summary,
                    return_tensors='pt',
                    max_length=1024,
                    truncation=True
                ).to(self.device)

                # Generate summary with BART (Abstractive step)
                generated_summary_ids = self.bart_model.generate(
                    **bart_input,
                    num_beams=4,
                    max_length=150,
                    early_stopping=True
                )

                generated_summary = self.bart_tokenizer.decode(generated_summary_ids[0], skip_special_tokens=True)

                predictions.append(generated_summary)
                references.append(labels[0])

        # Compute ROUGE scores
        rouge_scores = rouge.compute(predictions=predictions, references=references)
        print(f"ROUGE Scores: {rouge_scores}")


In [ ]:
import torch
from transformers import XLNetTokenizer, XLNetForSequenceClassification, BartTokenizer, BartForConditionalGeneration
from transformers import AdamW
from torch.cuda.amp import GradScaler, autocast
from evaluate import load  # Use evaluate library to load metrics
import numpy as np
import math

In [ ]:
# Initialize the model
model = Summ_xlnet_bart()

# Train the model (with mixed precision)
model.train(train_dataloader, val_dataloader, epochs=3)

# Evaluate the model on the validation set (ROUGE scores)
model.evaluate(val_dataloader)


Using device: cuda


Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-10-82f2478bfcca>:30: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = GradScaler()


Epoch 1/3


Training:   0%|          | 0/2500 [00:00<?, ?it/s]<ipython-input-10-82f2478bfcca>:73: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Avg Training Loss: 0.0016


ROUGE Scores: {'rouge1': 0.362444757950524, 'rouge2': 0.14954083104046387, 'rougeL': 0.24658175740327812, 'rougeLsum': 0.3189338528884851}
Epoch 2/3


Avg Training Loss: 0.0000
ROUGE Scores: {'rouge1': 0.3668806537029185, 'rouge2': 0.15549341742477812, 'rougeL': 0.24913322843602848, 'rougeLsum': 0.32097793513682654}
Epoch 3/3


Avg Training Loss: 0.0000
ROUGE Scores: {'rouge1': 0.36586649692629897, 'rouge2': 0.15714092028304133, 'rougeL': 0.2519961246800201, 'rougeLsum': 0.32335773766334525}
ROUGE Scores: {'rouge1': 0.36586649692629897, 'rouge2': 0.15714092028304133, 'rougeL': 0.2519961246800201, 'rougeLsum': 0.32335773766334525}


In [ ]:
model_state = {
    'xlnet_state': model.xlnet_model.state_dict(),
    'bart_state': model.bart_model.state_dict()
}
torch.save(model_state, "/summ_xlnet_bart.pth")
print(f"Model saved to /summ_xlnet_bart.pth")


Model saved to /summ_xlnet_bart.pth


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

torch.save(model_state, '/content/drive/My Drive/summ_xlnet_bart.pth')
print("Model saved to /content/drive/My Drive/summ_xlnet_bart.pth")


Model saved to /content/drive/My Drive/summ_xlnet_bart.pth
